In [58]:
SAMPLES_DIR="../samples/"
ETC_DIR=SAMPLES_DIR+"etc/"
AUDIO_DIR=SAMPLES_DIR+"audio/"
OUT_DIR=SAMPLES_DIR+"out/"
VIDEO_NAME="video.mp4"
AUDIO_NAME="video.wav"

In [59]:
from pydub import AudioSegment

video = AudioSegment.from_file(SAMPLES_DIR+IN_NAME, format="mp4")

In [60]:
audio = video.set_channels(1).set_frame_rate(16000).set_sample_width(2)

In [64]:
audio.export(AUDIO_DIR+AUDIO_NAME, format="wav")

<_io.BufferedRandom name='../samples/out/video.wav'>

In [65]:
import whisper
model = whisper.load_model("base")

In [133]:
output = model.transcribe(AUDIO_DIR+AUDIO_NAME)
print(output)

/opt/miniconda3/envs/sportlight/lib/python3.9/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [135]:
positive = list()

add = ["Straight", "biggie", "Cover", "OnDrive", "Square", "Forward", "stadium", "Defence", "Sweep", "Reverse",
           "FrontFoot ", "LegGlance ", "BackFoot", "SquareCut", "Pull ", "Shot", "Hook", "Uppercut", "Cut", "Helicopter ", "SwitchHit",
           "Dilscoop", "class", "bounce", "Upper", "Uppish", "Scoop ", "Inside", "Out", "Shots", "Bouncer", "Outswinger", "Inswinger",
           "ReverseSwing", "played", "LegCutter", "OffCutter", "Yorker", "Slower", "Spin", "LegBreak ", "OffBreak", "Googly ",
           "Doosra", "Topspin ", "CarromBall", "Slider", "ArmBall", "Infield", "InnerRing", "Outfield", "Catching", "Wicketkeeper",
           "Slip", "Gully", "LegSlip", "LegGully", "Sillypoint", "Sillymidoff", "Shortleg", "Sillymidon", "InnerRing", "Point", "BackwardPoint",
           "MidOff", "Cover", "MidOn", "SquareLeg", "Backward ", "SquareLeg", "MidWicket", "FineLeg", "Outfield", "ThirdMan",
           "DeepPoint", "BackwardPoint", "ExtraCover", "LongOff", "FineLeg", "LongLeg", "LongOn", "Deep", "Cover", "played", "account"
           "cricket", "hard", "sides", "man", "finishes", "one", "crucial", "Captain", "shot", "six", "four", "boundary", "line", "drive",
           "celebrate", "placement", "beauty", "fifty", "century", "perfect", "magnifcient", "world", "cup", "batting", "fielding", "bowling",
           "catch", "caught", "out", "stumped", "one", "bowled", "night", "final", "room", "taken", "edged", "wicket", "review", "DRS", "cuts", "out", "short"]

In [136]:
import nltk

nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/arjunsivaraman/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/arjunsivaraman/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [137]:
from nltk.corpus import wordnet

for i in add:
        for synset in wordnet.synsets(i):
            for lemma in synset.lemmas():
                positive.append(lemma.name())

In [138]:
strings = ' '.join(positive)

In [139]:
data = output['segments']

In [140]:
text = []
start_time = []
end_time = []

for i in data:
    text.append(i['text'])
    start_time.append(i['start'])
    end_time.append(i['end'])

In [142]:
import pandas as pd

def create_dataframe(matrix, tokens):
    doc_names = [f'doc_{i+1}' for i, _ in enumerate(matrix)]
    df = pd.DataFrame(data=matrix, index=doc_names, columns=tokens)
    return (df)

In [143]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

score = []
for i in text:
    d = [i, strings]
    Tfidf_vect = TfidfVectorizer()
    vector_matrix = Tfidf_vect.fit_transform(d)
    tokens = Tfidf_vect.get_feature_names_out()
    create_dataframe(vector_matrix.toarray(), tokens)
    cosine_similarity_matrix = cosine_similarity(vector_matrix)
    result = create_dataframe(
        cosine_similarity_matrix, ['Phrase', 'Strings'])
    score.append(result['Phrase'].values[1])

In [169]:
stack_first = []
stack_last = []

for i in range(len(score)):
    if (score[i] >= 0.00500000000000):
        stack_first.append(start_time[i])
        stack_last.append(end_time[i])

In [170]:
# Sort both lists based on one list
# stack_first, stack_last = zip(*sorted(zip(stack_first, stack_last)))
result = sorted(zip(stack_first, stack_last))

In [171]:
result

[(0.0, 8.0),
 (37.0, 45.0),
 (45.0, 49.0),
 (49.0, 51.0),
 (51.0, 55.0),
 (61.0, 64.0),
 (73.0, 81.0),
 (83.0, 89.0),
 (135.0, 139.0),
 (166.0, 171.0),
 (171.0, 180.0),
 (185.0, 193.0),
 (198.0, 203.0),
 (217.0, 223.0),
 (228.0, 234.0),
 (239.0, 243.0),
 (246.0, 252.0)]

In [172]:
res = [[result[0][0] - 4 if result[0][0] - 4 > 0 else 0, result[0][1] + 3]]
for i in range(1, len(result)):
    if res[-1][1] + 3 >= result[i][0] - 4:
        if res[-1][1] + 3 < result[i][1] + 3:
            res[-1][1] = result[i][1] + 3
    else:
        res.append([result[i][0] - 4 if result[i][0] - 4 > 0 else 0, result[i][1] + 3])

In [173]:
res

[[0, 11.0], [33.0, 92.0], [131.0, 142.0], [162.0, 206.0], [213.0, 255.0]]